# <center> Baseline ResNet

Nous allons utiliser ResNet pour nous créer une baseline de comparaison, mais également pour comparer les performances avec la partie vision.

## Imports de base

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

In [26]:
import sys
from pathlib import Path

# Pour que notre archi fonctionne avec google colab 
    
!git clone https://github.com/julietteabalain-cloud/Reconnaissance-de-mouvement-artistique.git
!cd /content/Reconnaissance-de-mouvement-artistique && git pull
%cd /content/Reconnaissance-de-mouvement-artistique 
import sys
sys.path.append(".")  # pour que src/ soit importable

PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))
DATA_ROOT = PROJECT_ROOT / "data"

fatal: destination path 'Reconnaissance-de-mouvement-artistique' already exists and is not an empty directory.
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 7 (delta 6), reused 7 (delta 6), pack-reused 0 (from 0)
Unpacking objects: 100% (7/7), 3.41 KiB | 1.14 MiB/s, done.
From https://github.com/julietteabalain-cloud/Reconnaissance-de-mouvement-artistique
   bafba22..1b8b998  main       -> origin/main
Updating bafba22..1b8b998
Fast-forward
 notebooks/4_RV-DL_BaselineResNet.ipynb | 118 +++++++++++++++++++++++++++++----
 src/dataset_dl.py                      |   2 +-
 src/evaluate.py                        |  19 ++++--
 3 files changed, 118 insertions(+), 21 deletions(-)
/content/Reconnaissance-de-mouvement-artistique


In [27]:
from src.dataset_dl import ArtDataset
from src.train import train_model, train_one_epoch, validate_one_epoch

from src.dataset import load_df_train_test_val, load_df
from src.preprocessing import clean_dataset

from src.models import get_resnet18
from src.evaluate import *
from src.utils import set_seed

#Fixer l'initialisation aléatoire pour la reproductibilité
set_seed(42)

#pour avoir acces au GPU si dispo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)



cuda


In [28]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path

PROJECT_ROOT = Path("/content/deepl-projet")
DATA_ROOT = Path("/content/drive/MyDrive/DeepLearning/WikiArt_Subset")


df_test, df_train, df_val = load_df_train_test_val(DATA_ROOT)
df = load_df(DATA_ROOT)

df, df_train, df_val, df_test = clean_dataset(df, df_train, df_val, df_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/content/Reconnaissance-de-mouvement-artistique/src/preprocessing.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["style_encoded"] = le.fit_transform(df_train["style"])
/content/Reconnaissance-de-mouvement-artistique/src/preprocessing.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val["style_encoded"]   = le.transform(df_val["style"])
/content/Reconnaissance-de-mouvement-artistique/src/preprocessing.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

## 1. Partie Classique / Baseline

### 1.1 Préparation du dataset

In [37]:
for i in range(5):
    print(f"Sample {i} - Style: {df_train.iloc[i]['style_name']} - Style number: {df_train.iloc[i]['style']}- Style code: {df_train.iloc[i]['style_encoded']}")

print(f"Number of unique styles: {df_train['style_name'].nunique()}")
print(f"Unique styles: {df_train['style'].unique()}")
print(f"Unique styles names: {df_train['style_encoded'].unique()}")

Sample 0 - Style: Expressionism - Style number: 9- Style code: 7
Sample 1 - Style: Fauvism - Style number: 10- Style code: 8
Sample 2 - Style: Symbolism - Style number: 24- Style code: 21
Sample 3 - Style: Abstract_Expressionism - Style number: 0- Style code: 0
Sample 4 - Style: Fauvism - Style number: 10- Style code: 8
Number of unique styles: 23
Unique styles: [ 9 10 24  0 11  5 26  6 17 18 21 22 19  7 20 12  4 13  8 23 15  3 14]
Unique styles names: [ 7  8 21  0  9  3 22  4 14 15 18 19 16  5 17 10  2 11  6 20 13  1 12]


On ajoute de la data augmentation pour éviter le surapprentissage.

In [38]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ), 
    # ajout de data augmentation pour le training set
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ), 
])


In [39]:
IMAGE_ROOT_TRAIN = DATA_ROOT / "train"
IMAGE_ROOT_VAL = DATA_ROOT / "val"
IMAGE_ROOT_TEST = DATA_ROOT / "test"

train_dataset = ArtDataset(
    df_train,
    IMAGE_ROOT_TRAIN,
    transform=transform_train
)

val_dataset = ArtDataset(
    df_val,
    IMAGE_ROOT_VAL,
    transform=transform_val
)

test_dataset = ArtDataset(
    df_test,
    IMAGE_ROOT_TEST,
    transform=transform_val
)



In [40]:
BATCH_SIZE = 32  

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,     # ajuster selon ton CPU
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,     # ajuster selon ton CPU
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,     # ajuster selon ton CPU
    pin_memory=True
)

### 1.2 Charger le modèle

In [41]:
num_classes = df_train["style"].nunique()
print("Min label:", df_train["style"].min())
print("Max label:", df_train["style"].max())
print("Nombre de classes uniques:", df_train["style"].nunique())


print(str(df_train["style"].nunique()))
model = get_resnet18(num_classes=num_classes, device=device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(
    model.fc.parameters(),
    lr=1e-3
)


Min label: 0
Max label: 26
Nombre de classes uniques: 23
23


### 1.3 Entrainement du modèle

On commence avec 3 epoch et on freeze.

In [42]:
row = df_train.iloc[0]

print(row["filename"])
print(row["style"])
print(row["style_encoded"])

print(row["style_name"])
print(df_train[["style","style_name"]].head(10))

print((IMAGE_ROOT_TRAIN / str(row["style"]) / row["filename"]).exists())
print((IMAGE_ROOT_TRAIN / str(row["style_encoded"]) / row["filename"]).exists())

2289.jpg
9
7
Expressionism
    style              style_name
1       9           Expressionism
2      10                 Fauvism
4      24               Symbolism
5       0  Abstract_Expressionism
6      10                 Fauvism
7      11        High_Renaissance
8       5    Color_Field_Painting
9      26                 Ukiyo_e
10      6    Contemporary_Realism
11      9           Expressionism
True
False


In [43]:
history_freeze = train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    num_epochs=3
)



Epoch 1/3


KeyboardInterrupt: 

unfreeze complet :

In [ ]:
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(
    model.parameters(),
    lr=1e-4
)


On continue avec cette fois 10 epoch.

In [ ]:
history_unfreeze = train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    num_epochs=10
)


### 1.4 Evaluation du modèle

#### 1.4.1 Evaluation train val

Accuracy (précision)  globale :

In [ ]:
train_acc = history_freeze["train_acc"] + history_unfreeze["train_acc"]
val_acc   = history_freeze["val_acc"]   + history_unfreeze["val_acc"]

plt.plot(train_acc)
plt.plot(val_acc)
plt.legend(["Train", "Validation"])
plt.title("ResNet18 Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()


Matrice de confusion :

In [ ]:
#Confusion matrix :

class_names = sorted(df_train["style_name"].unique())

cm = compute_confusion_matrix(
    model,
    val_loader,
    device,
    class_names
)

plot_confusion_matrix(cm, class_names)


Accuracy par style :

In [ ]:
class_names = sorted(df_train["style"].unique())

acc_per_style = accuracy_per_class(
    model,
    val_loader,
    device,
    class_names
)

results = list(zip(class_names, acc_per_style))
results = sorted(results, key=lambda x: x[1], reverse=True)

for style, acc in results:
    print(f"{style}: {acc:.3f}")


In [ ]:
visualize_accuracy_per_style(results)

#### 1.4.2 Evaluation test

In [ ]:
best_model_weights = model.state_dict()

test_acc, test_cm, report = evaluate_model(model, test_loader, device)

print(f"Test Accuracy: {test_acc:.3f}")
print("Classification Report:")
print(report)

### 1.5 Second modèle ResNet18

Surapprentissage très fort; modèle utilisé : Freeze 3 epochs, Unfreeze tout 10 epochs à 1e-4
On change de stratégie : 
Early stopping, dropout, unfreeze unqiuement layer 4, label smoothing.

In [ ]:
model_02 = get_resnet18(num_classes=num_classes, device=device, drop=True)

# ajout de label smoothing pour la cross entropy loss
# label smoothing permet de rendre le modèle moins confiant dans ses prédictions, 
# ce qui peut aider à améliorer la généralisation et réduire le surapprentissage
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = torch.optim.Adam(
    model_02.fc.parameters(),  
    lr=1e-3,
    weight_decay=1e-4
)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=3):
        self.patience = patience
        self.best_loss = float("inf")
        self.counter = 0
        self.stop = False

    def step(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.stop = True

In [ ]:
# from src.utils import EarlyStopping

early_stopping = EarlyStopping(patience=3)

NUM_EPOCHS_FREEZE = 3
history_freeze_02 = train_model(
    model_02,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    num_epochs=NUM_EPOCHS_FREEZE,
    early_stopping=early_stopping  
)

# Débloquer layer4 et fc pour le fine-tuning
for param in model_02.parameters():
    param.requires_grad = False
for param in model_02.layer4.parameters():
    param.requires_grad = True
for param in model_02.fc.parameters():
    param.requires_grad = True

optimizer = optim.Adam(
    model_02.parameters(),
    lr=1e-4, 
    weight_decay=1e-4)

NUM_EPOCHS_UNFREEZE = 7

history_unfreeze_02 = train_model(
    model_02,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    num_epochs=NUM_EPOCHS_UNFREEZE,
    early_stopping=early_stopping
)



### 1.6 Evaluation du nouveau modèle 

#### 1.6.1 Evaluation avec train + val

In [ ]:
train_acc = history_unfreeze_02["train_acc"] + history_freeze_02["train_acc"]
val_acc   = history_unfreeze_02["val_acc"] + history_freeze_02["val_acc"]

plt.plot(train_acc)
plt.plot(val_acc)
plt.legend(["Train", "Validation"])
plt.title("ResNet18 Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
#Confusion matrix :

class_names = sorted(df_train["style"].unique())

cm = compute_confusion_matrix(
    model_02,
    val_loader,
    device,
    class_names
)

plot_confusion_matrix(cm, class_names)


In [ ]:
class_names = sorted(df_train["style"].unique())

acc_per_style = accuracy_per_class(
    model_02,
    val_loader,
    device,
    class_names
)

results = list(zip(class_names, acc_per_style))
results = sorted(results, key=lambda x: x[1], reverse=True)

for style, acc in results:
    print(f"{style}: {acc:.3f}")


In [ ]:
visualize_accuracy_per_style(results)

#### 1.6.2 Evaluation sur l'ensemble de test

In [ ]:
best_model_weights = model_02.state_dict()

test_acc, test_cm, report = evaluate_model(model_02, test_loader, device)

print(f"Test Accuracy: {test_acc:.3f}")
print("Classification Report:")
print(report)

In [ ]:
print("Test Confusion Matrix:")
plot_confusion_matrix(test_cm, class_names)

### 1.5 Sauvegarde du modèle entrainé

In [ ]:
import os
SAVE_DIR = "/content/drive/MyDrive/models"
model_name_02 = "resnet18_baseline_02.pt"
os.makedirs("/content/drive/MyDrive/models", exist_ok=True)

save_path = os.path.join(SAVE_DIR, model_name_02)

torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "num_classes": num_classes,
    "learning_rate": 1e-3,
    "weight_decay": 1e-4,
    "batch_size": train_loader.batch_size,
    "num_epochs": len(history_02["train_loss"]),
    "history": history_02,
    "architecture": "resnet18_baseline_frozen_dropout",
}, save_path)

# Pour charger le modèle plus tard :
# checkpoint = torch.load(save_path, map_location=device)

# model = get_resnet18(
#     num_classes=checkpoint["num_classes"],
#     device=device
# )

# model.load_state_dict(checkpoint["model_state_dict"])

# model.eval()



🔹 Phase B – Préparer l’infrastructure pour ablation

 Créer fonction apply_low_pass(image)

 Créer fonction apply_high_pass(image)

 Version dataset avec filtrage optionnel

 Réentraîner ResNet sur chaque version

 Comparer accuracy globale

 Comparer accuracy par style

 Graphique comparatif final